# Extension - RRT: basic finding example
* Path finding is using sample-based (RRT) methods rather than constrained optimization to find a collision free path
* Path finding is much easier if a final robot pose is given. We here use IK (formulated as KOMO problem) to first compute a final configuration 'qT'. (Path optimization does this jointly with optimizing the path.)
* Then we can pass the current state and qT to a bi-directional RRT to find a collision free path.
* Note that BotOp (the interface to sim/real) is opened only after we computed the motion. We simply pass the motion to be played by the sim/real.

In [1]:
import robotic as ry
import time
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 3}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.helloworld()

Hello, World!


first a minimalistic example for testing:

In [ ]:
# # Create a new configuration
# C = ry.Config()

# C.addFile("/home/monke/Xrai/maze.g")

# # Add the moving "ego" frame with constraints on its motion
# C.addFrame("ego", "floorwalls") \
#     .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
#     .setRelativePosition([0.2, 0.2, 0.4]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([0, 1., 1.]) \
#     .setContact(1)

# C.addFrame("goal1") \
#     .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.addFrame("goal2") \
#     .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
#     .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
#     .setColor([1, 0, 0]) \

# C.view()
# q0 = [0.2, 0.2, 0.4]
# qT = C.getFrame('goal1').getPosition()

# ry.params_clear()
# ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 0,'rrt/maxIters':500}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

# rrt = ry.PathFinder()
# rrt.setProblem(C, [q0], [qT])
# ret = rrt.solve()
# print(ret)
# path = ret.x

# # Visualize the solution path
# if ret.feasible:
#     for i in range(len(path)):
#         C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02])  # Add a sphere at each path point
#         C.view()
# else:
#     print("No valid path found.")

{ time: 0.002588, evals: 30, done: 1, feasible: 1, eq: -1, ineq: -1, sos: -1, f: -1 }
entered RRT_Star_PathFinder
entered getPathFromNode
ann.X[0.2, 0.2, 0.4,
 0.151382, 0.167216, 0.318997,
 0.153077, 0.06727, 0.32182,
 0.154713, -0.0326795, 0.324546,
 0.15641, -0.132625, 0.327373,
 0.158198, -0.232608, 0.327785,
 0.194082, 0.224947, 0.496657,
 0.123287, 0.193316, 0.226642,
 0.159986, -0.332591, 0.328197,
 0.106564, 0.175386, 0.129694,
 0.167373, 0.223545, 0.593014,
 0.0576655, 0.214938, 0.0519472,
 0.29722, 0.184866, 0.382134,
 0.157994, -0.431651, 0.314658,
 0.0642198, 0.179699, -0.0414084,
 0.156002, -0.53071, 0.301119,
 0.158169, -0.63067, 0.302932,
 0.0301699, 0.162031, -0.133758,
 0.160204, -0.730535, 0.307711,
 0.165877, 0.244049, 0.690878,
 0.157475, -0.829116, 0.291145,
 0.159935, -0.929048, 0.293868,
 0.0328886, 0.115968, -0.222476,
 0.10832, -0.991842, 0.352115,
 0.0420372, 0.092535, -0.31926,
 -0.0167914, 0.0814919, -0.399368,
 0.0875564, -1.08881, 0.365039,
 -0.0907517, 0.

In [ ]:
# Create a new configuration
C = ry.Config()

C.addFile("/home/monke/Xrai/maze.g")

# Add the moving "ego" frame with constraints on its motion
C.addFrame("ego", "floorwalls") \
    .setJoint(ry.JT.transXYPhi, [-1., 1., -1., 1., -3., 3.]) \
    .setRelativePosition([0.2, 0.2, 0.4]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([0, 1., 1.]) \
    .setContact(1)

C.addFrame("goal1") \
    .setPosition(C.getFrame('ego').getPosition()+[0, -3, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

C.addFrame("goal2") \
    .setPosition(C.getFrame('goal1').getPosition()+[4, 0, 0]) \
    .setShape(ry.ST.ssBox, size=[0.05, 0.3, 0.05, 0.01]) \
    .setColor([1, 0, 0]) \

# C.view()
q0 = [0.2, 0.2, 0.4]
qT = C.getFrame('goal1').getPosition()

ry.params_clear()
ry.params_add({'rrt/stepsize':.08, 'rrt/verbose': 0,'rrt/maxIters':50}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

rrt = ry.PathFinder()
rrt.setProblem(C, [q0], [qT])
ret = rrt.star_solve()
print(ret)
path = ret.x

# Visualize the solution path
if ret.feasible:
    for i in range(len(path)):
        C.addFrame(f"path_{i}").setPosition(path[i]).setShape(ry.ST.sphere, [0.02])  # Add a sphere at each path point
        C.view()
else:
    print("No valid path found.")
